# Example - 2CDE Method

*This notebook is part of smFRET burst analysis software [FRETBursts](http://tritemio.github.io/FRETBursts/).*

> **Experimental!!! Implementation to be double-checked.** 
>
> This notebook implements the 2CDE method from [Tomov 2012](http://dx.doi.org/10.1016%2Fj.bpj.2011.11.4025).

# Load Data

In [ ]:
from fretbursts import *
from fretbursts.phtools import phrates
sns = init_notebook()

In [ ]:
import fretbursts as fb

In [ ]:
url = 'http://files.figshare.com/2182601/0023uLRpitc_NTP_20dT_0.5GndCl.hdf5'
download_file(url, save_dir='./data')

In [ ]:
filename = "data/0023uLRpitc_NTP_20dT_0.5GndCl.hdf5"

In [ ]:
d = loader.photon_hdf5(filename)
#bpl.plot_alternation_hist(d)
loader.alex_apply_period(d)
d.calc_bg(fun=bg.exp_fit, time_s=20, tail_min_us='auto', F_bg=1.7)
d.burst_search()

In [ ]:
ds1 = d.select_bursts(select_bursts.size, th1=20)
ds = ds1.select_bursts(select_bursts.naa, th1=20)

alex_jointplot(ds)

In [ ]:
ph = d.ph_times_m[0]

In [ ]:
tau = 100e-6/d.clk_p
tau

# KDE considerations

For start, let's compute the instantaneous photon-rate via KDE on a syntetic timestamps. We compare different kernels: laplace distribution (i.e. exponential), Gaussian and rectangular.

In [ ]:
tau = 1000

In [ ]:
t = np.arange(0, 14000, 1000, dtype=np.int64)
tx = np.arange(0, t.max(), 0.01*(t[1] - t[0]), dtype=np.int64)
true_rate = (t.size - 1) / t.max()

rate_rect = phrates.kde_rect(t, tau, tx)
rate_gauss = phrates.kde_gaussian(t, tau, tx)
rate_laplace = phrates.kde_laplace(t, tau, tx)

fig, ax = plt.subplots(figsize=(17, 4))
plot(tx, rate_laplace / (2*tau), label='exp')
plot(tx, rate_rect / tau, label='rect')
plot(tx, rate_gauss / (2.5*tau), label='gauss')
plt.axhline(true_rate, ls='--', color='k')
plt.legend(loc='lower center', fontsize=18)
sns.distplot(t, rug=True, kde=False, hist=False)

In [ ]:
t = np.arange(0, 14000, 2000, dtype=np.int64)
tx = np.arange(0, t.max(), 0.01*(t[1] - t[0]), dtype=np.int64)
true_rate = (t.size - 1) / t.max()

rate_rect = phrates.kde_rect(t, tau, tx)
rate_gauss = phrates.kde_gaussian(t, tau, tx)
rate_laplace = phrates.kde_laplace(t, tau, tx)

fig, ax = plt.subplots(figsize=(17, 4))
plot(tx, rate_laplace / (2*tau), label='exp')
plot(tx, rate_rect / tau, label='rect')
plot(tx, rate_gauss / (2.5*tau), label='gauss')
plt.axhline(true_rate, ls='--', color='k')
plt.legend(loc='lower center', fontsize=18)
sns.distplot(t, rug=True, kde=False, hist=False)

In [ ]:
tau = 1000
true_rate = 0.001
num_times = 14
time_max = num_times/true_rate

np.random.seed(1)
t = np.sort(np.random.randint(low=0, high=time_max, size=num_times))
tx = np.arange(0, time_max, 1/(30*true_rate), dtype=np.int64)

#r, n = phrates.nb.kde_laplace_nph(t, tau, tx)
rate_rect = phrates.kde_rect(t, tau*5, tx)
rate_gauss = phrates.kde_gaussian(t, tau, tx)
rate_laplace = phrates.kde_laplace(t, tau, tx)

fig, ax = plt.subplots(figsize=(17, 4))
plot(tx, rate_laplace / (2*tau), label='exp')
plot(tx, rate_rect / (5*tau), label='rect')
plot(tx, rate_gauss / (2.5*tau), label='gauss')
plt.axhline(true_rate, ls='--', color='k')
plt.legend(loc='lower center', fontsize=18)
sns.distplot(t, rug=True, kde=False, hist=False)

In [ ]:
tau = 1
tau2 = 2 * (tau**2)

xx = np.arange(-4*tau, 4*tau, tau/100)
y1 = np.exp(-np.abs(xx) / tau)
y2 = np.exp(-xx**2 / tau2)

plt.plot(xx,y1, label=r'$\exp \left( - \frac{|t|}{\tau} \right)$')
plt.plot(xx, y2, label=r'$\exp \left( - \frac{t^2}{2\tau^2} \right)$')
plt.axvline(2*tau, color='k')
plt.axvline(-2*tau, color='k')
plt.xlabel('t')
plt.legend(fontsize=22, bbox_to_anchor=(1.05, 1), loc=2)
plt.title(r'$\tau = %d$' % tau, fontsize=22);

## Notes on Kernel Shape

The Gaussian kernel gives a more accurate rate estimation with very little dependence on the position where the KDE is evaluated. On the contrary, with symmetric exponential kernel (laplace distribution), there is always a strong dependence on the evaluation position. In particular, when rates are estimated at the timestamps positions, the rates are systematically over-estimated (i.e. the peak is always samples).

For Gaussian kernel, given a $\tau$, the rate extimation will be accurate for rates higher than $1/(2\,\tau)$ counts-per-second. For lower rates, the estimation will strongly depend on where the KDE is evaluated. A similar condition can be also found for the exponential kernel, but this case the rate will aways be strongly dependent on the position.

# 2CDE

## KDE and nbKDE definitions

Following Tomov 2012 notation, we define *KDE* as ([Tomov 2012, eq. 4](http://dx.doi.org/10.1016%2Fj.bpj.2011.11.4025)):

$$KDE_{X_i}^Y \left(t_{(CHX)_i}, t_{\{CHY\}} \right) = 
\sum_j^{N_{CHY}} \exp \left( - \frac{\lvert t_{(CHX)_i} - t_{(CHY)_j} \rvert}{\tau}\right) $$

and *nbKDE* as ([Tomov 2012, eq. 5](http://dx.doi.org/10.1016%2Fj.bpj.2011.11.4025)):

$$nbKDE_{X_i}^X \left(t_{\{CHX\}} \right) = \left(1 + \frac{2}{N_{CHX}} \right) \cdot
\sum_{j, \;j\ne i}^{N_{CHX}} \exp \left( - \frac{\lvert t_{(CHX)_i} - t_{(CHX)_j} \rvert}{\tau}\right) $$

These quantities are computed on the entire timestamp arrays, without selection of burst regions. $N_{CHX}$, in the multiplicatice correction factor of nbKDE, is therefore the 
number of photons in the entire timestamps array (selecting only photons in the $X$ channel).

## FRET-2CDE definition

To compute FRET-2CDE we need to define ([Tomov 2012, eq. 6](http://dx.doi.org/10.1016%2Fj.bpj.2011.11.4025)):

$$(E)_D = \frac{1}{N_{CHD}} \sum_{i=1}^{N_{CHD}} \frac{KDE_{Di}^A}{KDE_{Di}^A + nbKDE_{Di}^D} $$

and the symmetric estimator ([Tomov 2012, eq. 7](http://dx.doi.org/10.1016%2Fj.bpj.2011.11.4025)):

$$(1 - E)_A = \frac{1}{N_{CHA}} \sum_{i=1}^{N_{CHA}} \frac{KDE_{Ai}^D}{KDE_{Ai}^D + nbKDE_{Ai}^A} $$

Then *FRET-2CDE* is defined as ([Tomov 2012, eq. 8](http://dx.doi.org/10.1016%2Fj.bpj.2011.11.4025)):

$$ FRET-2CDE \left( t_{CHD}, t_{CHA} \right) = 
110 - 100 \cdot \left[ (E)_D + (1 - E)_A \right]
$$

These quantities are computed for each burst, so that $N_{CHD}$ ($N_{CHA}$) are
the number of photons in the DexDem (AemDex) channel during current burst.

## FRET-2CDE Implementation

In [ ]:
tau_s = 50e-6
tau = int(tau_s/d.clk_p)
tau

In [ ]:
ph = d.get_ph_times(ph_sel=Ph_sel('all'))
mask_d = d.get_ph_mask(ph_sel=Ph_sel(Dex='Dem'))
mask_a = d.get_ph_mask(ph_sel=Ph_sel(Dex='Aem'))

bursts = ds.mburst[0]

In [ ]:
def calc_fret_2cde(tau, ph, mask_d, mask_a, bursts):
    """
    Compute FRET-2CDE for each burst.
    
    Reference: Tomov et al. BJ (2012) doi:10.1016/j.bpj.2011.11.4025
    
    Arguments:
        tau (scalar): time-constant of the exponential KDE
        ph (1D array): array of all-photons timestamps.
        mask_d (bool array): mask for DexDem photons
        mask_a (bool array): mask for DexAem photons
        bursts (Bursts object): object containing burst data 
            (start-stop indexes are relative to `ph`).
        
    Returns:
        FRET_2CDE (1D array): array of FRET_2CDE quantities, one element per burst.
    """
    # Computing KDE burst-by-burst would cause inaccuracies at the edges
    # So, we compute KDE for the full timestamps
    KDE_DTi = phrates.kde_laplace(ph[mask_d], tau, time_axis=ph)
    KDE_ATi = phrates.kde_laplace(ph[mask_a], tau, time_axis=ph)

    # nbKDE does not include the "center" timestamp which contributes 1.
    # Therefore we substract 1 from precomputed KDEs
    # The normalization factor contains the number or photons
    # in the entire photon stream used to compute the KDE.
    N_CHD = mask_d.sum()
    N_CHA = mask_a.sum()
    nbKDE_DTi = (1 + 2/N_CHD) * (KDE_DTi - 1)
    nbKDE_ATi = (1 + 2/N_CHA) * (KDE_ATi - 1)
    
    FRET_2CDE = []
    for ib, burst in enumerate(bursts):
        burst_slice = slice(int(burst.istart), int(burst.istop) + 1)

        kde_adi = KDE_ATi[burst_slice][mask_d[burst_slice]]
        nbkde_ddi = nbKDE_DTi[burst_slice][mask_d[burst_slice]]
        # Here I assume that N_CHD of eq. 6 in (Tomov 2012) is the 
        # number of DexDem photons in current burst, therefore the sum 
        # is a mean (over non-NaNs elements)
        ED = np.nanmean(kde_adi / (kde_adi + nbkde_ddi))  # (E)_D 
    
        kde_dai = KDE_DTi[burst_slice][mask_a[burst_slice]]
        nbkde_aai = nbKDE_ATi[burst_slice][mask_a[burst_slice]]
        # Likewise for N_CHA of eq. 7 in (Tomov 2012). 
        # See previous comment.
        EA = np.nanmean(kde_dai / (kde_dai + nbkde_aai))  # (1 - E)_A 

        fret_2cde = 110 - 100 * (ED + EA)
        FRET_2CDE.append(fret_2cde)
    return np.array(FRET_2CDE)

In [ ]:
def calc_fret_2cde_gauss(tau, ph, mask_d, mask_a, bursts):
    """
    Compute a modification of FRET-2CDE using a Gaussian kernel.
    
    Reference: Tomov et al. BJ (2012) doi:10.1016/j.bpj.2011.11.4025
    
    Instead of using the exponetial kernel (i.e. laplace distribution)
    of the original paper, here we use a Gaussian kernel which has
    better properties and avoids over-estimation when evaluating the
    KDE at the timestamps positions. This removes the need of using
    the euristic correction (pre-factor) in nbKDE.
    
    Arguments:
        tau (scalar): time-constant of the exponential KDE
        ph (1D array): array of all-photons timestamps.
        mask_d (bool array): mask for DexDem photons
        mask_a (bool array): mask for DexAem photons
        bursts (Bursts object): object containing burst data
        
    Returns:
        FRET_2CDE (1D array): array of FRET_2CDE quantities, one element per burst.
    """
    # Computing KDE burst-by-burst would cause inaccuracies at the edges
    # So, we compute KDE for the full timestamps
    KDE_DTi = phrates.kde_gaussian(ph[mask_d], tau, time_axis=ph)
    KDE_ATi = phrates.kde_gaussian(ph[mask_a], tau, time_axis=ph)

    FRET_2CDE = []
    for ib, burst in enumerate(bursts):
        burst_slice = slice(int(burst.istart), int(burst.istop) + 1)

        kde_ddi = KDE_DTi[burst_slice][mask_d[burst_slice]]
        kde_adi = KDE_ATi[burst_slice][mask_d[burst_slice]]
        ED = np.nanmean(kde_adi / (kde_adi + kde_ddi))
    
        kde_dai = KDE_DTi[burst_slice][mask_a[burst_slice]]
        kde_aai = KDE_ATi[burst_slice][mask_a[burst_slice]]
        EA = np.nanmean(kde_dai / (kde_dai + kde_aai))

        fret_2cde = 110 - 100 * (ED + EA)
        FRET_2CDE.append(fret_2cde)
    return np.array(FRET_2CDE)

In [ ]:
fret_2cde = calc_fret_2cde(tau, ph, mask_d, mask_a, bursts)

In [ ]:
fret_2cde_gauss = calc_fret_2cde_gauss(tau, ph, mask_d, mask_a, bursts)

In [ ]:
len(fret_2cde), bursts.num_bursts

In [ ]:
len(fret_2cde_gauss), bursts.num_bursts

In [ ]:
plt.plot(ds.E[0], fret_2cde, 'o', alpha=0.05)

In [ ]:
valid = np.isfinite(fret_2cde)
print('Number of bursts (removing NaNs):', valid.sum())
fig = plt.figure(figsize=(6,6))
sns.kdeplot(ds.E[0][valid], data2=fret_2cde[valid], cmap='viridis', shade=True, shade_lowest=False)
plt.xlabel('E')
plt.ylabel('FRET-2CDE');

In [ ]:
valid = np.isfinite(fret_2cde_gauss)
print('Number of bursts (removing NaNs):', valid.sum())
fig = plt.figure(figsize=(6,6))
sns.kdeplot(ds.E[0][valid], data2=fret_2cde_gauss[valid], cmap='viridis', shade=True, shade_lowest=False)
plt.xlabel('E')
plt.ylabel('FRET-2CDE');

In [ ]:
g = sns.jointplot(ds.E[0], fret_2cde, kind='hex', 
                  joint_kws={'cmap': 'viridis', 'mincnt': 1, 'gridsize': 40})
g.ax_joint.set_xlabel('E')
g.ax_joint.set_ylabel('FRET-2CDE');

In [ ]:
g = sns.jointplot(ds.E[0], fret_2cde_gauss, kind='hex', 
                  joint_kws={'cmap': 'viridis', 'mincnt': 1, 'gridsize': 40})
g.ax_joint.set_xlabel('E')
g.ax_joint.set_ylabel('FRET-2CDE');

In [ ]:
alex_jointplot(ds)